## Plotting Robert's data

This notebook will plot our bootstrapped version of Robert's data.

This expects that there is a file `bootstrapped_ecs_normalized.parquet` in `data`.


In [ ]:
from os.path import join as pjoin
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def melt(df):
    melt_cols = ["human-human-ec", "human-machine-ec", "machine-machine-ec"]
    return df.melt(
        id_vars=[col for col in df.columns if col not in melt_cols],
        value_vars=melt_cols,
        var_name="comp_type",
        value_name="kappa",
    )

In [ ]:
# loading our bootstrap-results
standard_df = pd.read_parquet(
    pjoin("data", f"bootstrapped_ecs_standard.parquet"), engine="pyarrow"
)
standard_df = melt(standard_df)

normalized_df = pd.read_parquet(
    pjoin("data", f"bootstrapped_ecs_normalized.parquet"), engine="pyarrow"
)
normalized_df = melt(normalized_df)

In [ ]:
xlabel_ordering = {
    "low-pass": ["0", "1", "3", "5", "7", "10", "15", "40"],
    "phase-scrambling": ["0", "30", "60", "90", "120", "150", "180"],
    "rotation": ["0", "90", "180", "270"],
    "colour": ["cr", "bw"],
    "uniform-noise": ["0.0", "0.03", "0.05", "0.1", "0.2", "0.35", "0.6", "0.9"],
    "false-colour": ["True", "False"],
    "power-equalisation": ["0", "pow"],
    "contrast": ["c100", "c50", "c30", "c15", "c10", "c05", "c03", "c01"],
    "high-pass": [np.inf, "3.0", "1.5", "1.0", "0.7", "0.55", "0.45", "0.4"],
    "eidolon-I": ["0", "1", "2", "3", "4", "5", "6", "7"],
    "eidolon-II": ["0", "1", "2", "3", "4", "5", "6", "7"],
    "eidolon-III": ["0", "1", "2", "3", "4", "5", "6", "7"],
}

In [ ]:
# recreating Robert's plots of EC
experiments = standard_df["experiment"].unique()  # 12 experiments total

# representative experiments selected for main figure
selected_experiments = [
    "colour",
    "false-colour",
    "rotation",
]
# uncomment this to see all
# selected_experiments = experiments

fig, axes = plt.subplots(
    len(selected_experiments),
    2,
    figsize=(12, 4 * len(selected_experiments)),
    sharey=True,
)


def plot_column(df, ax_col, title_suffix):
    idx = 0
    for experiment, exp_df in df.groupby("experiment", observed=True):

        # skip all experiments that weren't selected
        if experiment not in selected_experiments:
            continue

        ax = ax_col[idx]

        ax.grid(axis="y")
        sns.pointplot(
            data=exp_df[exp_df["comp_type"] != "machine-machine-ec"],
            x="condition",
            y="kappa",
            hue="comp_type",
            order=(
                xlabel_ordering[experiment] if experiment in xlabel_ordering else None
            ),
            hue_order=["human-human-ec", "human-machine-ec"],
            dodge=True,
            palette="mako",
            estimator=np.mean,
            errorbar=("pi", 95),
            capsize=0.2,
            legend=idx == 0,
            ax=ax,
        )
        sns.despine(ax=ax)
        ax.set_title(f"{experiment} {title_suffix}")

        if idx == 0:
            legend = ax.get_legend()
            legend.set_title("EC type")
        ax.set_ylabel("EC (Kappa)")
        ax.set_xlabel("Condition")
        # ax.set_ylim(-0.2, 1.0)
        ax.set_ylim(0, 0.7)
        idx += 1


# plot left and right column
plot_column(standard_df, axes[:, 0], "(standard EC)")
plot_column(normalized_df, axes[:, 1], "(normalized EC)")

plt.tight_layout()
plt.savefig(
    pjoin("figures", f"geirhos_bootstrapped_selected_ecs.pdf"), bbox_inches="tight"
)
plt.show()